In [ ]:
import pandas as pd
from dmyplant2 import cred, MyPlant, Engine, FSMOperator
import dplayFSM3
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();

In [ ]:
e=Engine.from_sn(mp,1486174)
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
e['IB Site Name'] + ' ' + e['Engine ID'], modes, success

In [ ]:
e=Engine.from_sn(mp,1486174)
fsm = FSMOperator(e, p_from='2022-01-26', p_to='2022-02-01', successtime=600)
#fsm = dplayFSM3.demoFSM(e, p_to='2022-03-10', successtime=600)
fsm.run1(enforce=True) # run Finite State Machine

In [ ]:
rda = fsm.starts
rda['stime'] = rda.apply(lambda x: x['starttime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda['etime'] = rda.apply(lambda x: x['endtime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda[['no','success','mode','stime','etime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','targetoperation','rampdown','coolrun','runout']].round(2).fillna('-')

In [ ]:
rdc = fsm.stops
rdc[['no','mode','starttime','endtime','count_alarms','count_warnings']].round(2).fillna('-')

In [ ]:
fsm._runlogdetail;

In [ ]:
fsm.starts.iloc[4].to_dict()

In [27]:
############ START BOILERPLATE ############
#### Interactivity -- BOKEH
import os 
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "0f119f0o0ovb6puo7dn7u7kcngpu87ucdle4ccq6381miu3psuiu"
import bokeh
import bokeh.plotting.figure as figure
from bokeh.io import curdoc, show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput, Button, CheckboxGroup
from bokeh.io import output_notebook # enables plot interface in J notebook
import numpy as np
# init bokeh

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler


output_notebook()
############ END BOILERPLATE ############

# Set up data
N = 200
x = np.linspace(0, 4*np.pi, N)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))

# Set up plot
plot = figure(plot_height=600, plot_width=800, title="my sine wave",
              tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

# Set up widgets

#LABELS = ['Timings','Warnings','Alarms','Devices']
#checkbox_group = CheckboxGroup(labels=LABELS, active=[0, 1])
text = TextInput(title="title", value='my sine wave')
startversuch = Slider(title='Startversuch', value=0, start=0, end=1277, step=1)
offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)
but_up = Button(label='^', button_type='primary', width=50)
but_down = Button(label='v', button_type='warning', width=50)

# Set up callbacks
def update_title(attrname, old, new):
    plot.title.text = text.value

def update_up(event):
    startversuch.value += 1

def update_down(event):
    startversuch.value -= 1

def update_data(attrname, old, new):
    # Get the current slider values
    a = amplitude.value
    b = offset.value
    w = phase.value
    k = freq.value
    sv = startversuch.value

    # Generate the new curve
    x = np.linspace(0, 4*np.pi, N)
    y = a*np.sin(k*x + w) + b

    source.data = dict(x=x, y=y)
    ### I thought I might need a show() here, but it doesn't make a difference if I add one
    # show(layout)

#def update_checkbox(active):
#    plot.title.text = 'checkbox'

for w in [startversuch, offset, amplitude, phase, freq]:
    w.on_change('value', update_data)

but_up.on_click(update_up)
but_down.on_click(update_down)

#checkbox_group.on_change('name', update_checkbox)

# Set up layouts and add to document
#inputs = bokeh.models.Column(text, offset, amplitude, phase, freq, but_up, but_down)
layout = row(plot, bokeh.models.Column(text, startversuch, offset, amplitude, phase, freq, row(but_up, but_down)))


def modify_doc(doc):
    doc.add_root(row(layout, width=800))
    doc.title = "Sliders"
    text.on_change('value', update_title)

handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

Loading BokehJS ...